In [1]:
import numpy as np
import pandas as pd
import tokenize
import naming as naming
import suji as sj
from crawling import getText, getOneText
import io
from time import time
import ast

In [2]:

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)


users_link = 'http://114.71.103.80/new_db.php?q=select+*+from+users%0D%0Alimit+50%0D%0A%0D%0A'
def code_link(user):
    before = 'http://114.71.103.80/new_db.php?q=select+*+from+code_submission+c%2C+%28select+id%2C+max%28regtime%29+a+from+code_submission+where+id+%3D'
    after = '+group+by+p_no%29+d+where+c.id+%3D+d.id+and+d.a+%3D+c.regtime+'
    return before + user + after

total_num_p = 78


def make_feature_df(feature_func, total_num_p = total_num_p):
    users = getText(users_link, '1')
    df = pd.DataFrame(columns=users,index=range(1,total_num_p + 1))
    start = time()
    for user in users:
        num_p = getText(code_link(user),  '2')
        num_p = list(map(int, num_p))
        problems = getText(code_link(user),  '4')
        results=[]
        for problem in problems:
            stream = io.StringIO(problem)
            tokens = tokenize.generate_tokens(stream.readline)
            try:
                results.append(feature_func(tokens))
            except:
                results.append(None) # None = error in code

        result_i = 0
        for i, v in enumerate(num_p):
            df.loc[v][user] = results[i]
        # NaN = problem isn't submitted
    end = time()
    print(f'Running time {end - start} sec')
    return df

def avg_cleaning(df): #suji
    df.loc[:,'mean'] = df.mean(axis=1)
    df = df.divide(df['mean'], axis = 0)
    df = df.drop(['mean'], axis=1)
    return df

def self_cleaning(df): # yeram
    num_p, num_u = df.shape
    for i in range(1, num_p + 1):
        for j in range(num_u):
            lst=df.loc[i][j]
            if type(lst) != list:
                pass
            else:
                last_num=len(lst)-1
                for k in range(len(lst)):
                    if lst[last_num] != 0:
                        lst[k]=lst[k]/lst[last_num]
                del lst[last_num]
                df.loc[i][j]=lst
    return df


def integrate(dfs, prob, user): #seulin
    def check_shape(dfs): # check the shape of all dfs
        shape = dfs[0].shape
        for df in dfs:
            assert(shape == df.shape)
    check_shape(dfs)
    res = []
    for df in dfs:
        feature = df.loc[prob][user]
        if type(feature) == list:
            res.extend(feature)
        elif type(feature) in [int, float]:
            res.append(feature)
        else:
            # NaN
            pass
    return res

In [3]:
df1 = make_feature_df(naming.naming_variances)
df1 = self_cleaning(df1)
df1

Running time 119.57375359535217 sec


,19900287,201411023,201411085,201411130,201411153,201511020,201511057,201511080,201511153,201611074,201611101,201611104,201611155,201611161,201611172,201711036,201711084,201711091,201711094,201811003,201811004,201811017,201811032,201811034,201811046,201811048,201811054,201811075,201811076,201811087,201811090,201811098,201811103,201811105,201811121,201811125,201811140,201811141,201811143,201811145,201811146,201811147,201811148,201811157,201811172,201811181,201811191,201811193,201822019,201822021
1,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.029411764705882353, 0.0294117647058823...","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]"
2,"[0.0, 0.034482758620689655, 0.0344827586206896...","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,NaN,"[0.0, 0.0, 0.0, 1.0]",NaN,NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]"
3,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.5, 0.0, 0.0, 1.0]","[0.5, 0.0, 0.0, 1.0]","[0.5, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]",NaN,"[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]",NaN
4,"[0.0, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]","[0.2, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]",NaN,"[0.4, 0.0, 0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]","[0.2, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]","[0.4, 0.0, 0.0, 1.0]","[0.25, 0.0, 0.

In [4]:
df2 = make_feature_df(sj.Total_Line)
df2 = avg_cleaning(df2)
df2

Running time 116.71864032745361 sec


,19900287,201411023,201411085,201411130,201411153,201511020,201511057,201511080,201511153,201611074,201611101,201611104,201611155,201611161,201611172,201711036,201711084,201711091,201711094,201811003,201811004,201811017,201811032,201811034,201811046,201811048,201811054,201811075,201811076,201811087,201811090,201811098,201811103,201811105,201811121,201811125,201811140,201811141,201811143,201811145,201811146,201811147,201811148,201811157,201811172,201811181,201811191,201811193,201822019,201822021
1,3.74847,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,NaN,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,1.73006,NaN,0.288344,NaN,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,0.288344,18.1656,10.9571,0.288344,0.288344,0.288344
2,4.07692,1,1,NaN,1,1,NaN,NaN,1,NaN,NaN,1,1,NaN,1,0.153846,1,0.0769231,1,NaN,1,NaN,1,NaN,1,NaN,1,1.07692,1,1,1,1,NaN,1,1,1,1,0.384615,1,1,NaN,1,1,1,1,0.230769,1,NaN,1,1
3,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,NaN,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,NaN,0.958333,NaN,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,1.4375,0.958333,0.958333,0.958333,1.4375,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,1.4375,1.4375,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,0.958333,NaN
4,1.94366,0.971831,1.29577,0.971831,0.971831,0.971831,1.29577,0.971831,0.971831,0.971831,NaN,0.971831,0.971831,0.971831,0.971831,0.971831,0.971831,0.971831,0.971831,NaN,0.971831,NaN,1.29577,1.29577,0.971831,0.971831,0.647887,0.971831,0.971831,0.647887,0.971831,0.971831,1.29577,0.971831,0.323944,0.647887,0.971831,0.971831,0.971831,0.971831,0.971831,0.971831,0.647887,0.647887,0.971831,1.29577,1.29577,1.29577,0.971831,NaN
5,1.34853,0.899023,1.1987,0.899023,1.1987,0.749186,1.34853,0.749186,0.899023,0.899023,NaN,0.899023,0.899023,0.899023,0.899023,0.899023,1.34853,1.1987,NaN,NaN,0.899023,NaN,0.749186,1.1987,1.34853,0.899023,0.899023,0.899023,1.34853,0.899023,0.899023,0.749186,1.94788,1.34853,0.149837,0.749186,1.04886,0.599349,1.49837,1.34853,1.34853,0.749186,0.749186,0.749186,0.749186,0.899023,0.899023,1.34853,0.899023,0.899023
6,1.33818,1.00364,1.17091,1.00364,1.17091,0.669091,1.17091,1.00364,1.00364,1.00364,NaN,1.00364,1.00364,1.00364,1.00364,0.836364,1.00364,1.17091,0.501818,NaN,1.00364,NaN,1.17091,1.00364,0.669091,0.836364,1.33818,1.00364,1.17091,0.836364,1.00364,0.669091,1.67273,1.17091,0.669091,1.00364,0.836364,0.836364,1.00364,1.17091,1.17091,0.836364,0.669091,1.00364,1.00364,1.00364,1.00364,1.17091,1.00364,NaN
7,0.806723,0.537815,1.21008,1.34454,0.806723,0.941176,1.61345,0.672269,0.672269,1.07563,NaN,1.07563,1.07563,0.941176,0.672269,0.672269,0.672269,0.806723,1.47899,NaN,1.34454,0.672269,1.07563,0.806723,1.21008,0.806723,1.47899,0.672269,1.34454,0.672269,0.941176,0.672269,1.34454,0.672269,0.672269,1.07563,0.806723,1.34454,2.01681,1.21008,1.34454,0.941176,1.21008,0.941176,1.07563,1.07563,0.672269,1.07563,0.672269,1.07563
8,0.907563,1.51261,1.0084,1.0084,1.0084,1.21008,1.10924,1.0084,1.0084,0.907563,NaN,0.907563,0.907563,0.907563,0.907563,1.0084,0.907563,1.10924,1.0084,NaN,1.10924,0.907563,0.907563,1.0084,1.0084,1.0084,1.31092,0.907563,0.907563,0.907563,1.10924,0.907563,1.51261,0.907563,1.0084,0.907563,1.41176,0.907563,0.907563,1.0084,0.907563,0.907563,1.0084,1.0084,0.907563,0.907563,0.907563,1.10924,0.907563,0.504202
9,0.798226,0.997783,1.09756,NaN,1.19734,0.798226,1.09756,0.997783,1.89579,0.997783,NaN,0.997783,0.997783,0.798226,1.19734,0.798226,0.798226,0.997783,1.59645,NaN,0.798226,0.798226,0.997783,1.09756,1.09756,1.09756,0.798226,0.997783,0.997783,0.798226,1.3969,0.798226,1.59645,0.997783,0.798226,0.798226,0.898004,0.798226,0.798226,0.997783,0.997783,0.997783,1.19734,0.798226,0.798226,0.997783,0.997783,NaN,0.798226,NaN
10,0

In [5]:
integrate([df1, df2], 10, "201411023")

[0.0, 0.0, 0.0, 1.0, 1.153846153846154]

In [ ]:
# save and read
df.to_csv("feature_df.csv")
df_data = pd.read_csv("feature_df.csv", index_col = 0)
# dic = {range(50):ast.literal_eval}
# dff = pd.read_csv("feature_df.csv", index_col = 0, converters = {0:ast.literal_eval})